# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

In [1]:
##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *

Loading ras-commander from local dev copy


# Delineate Fluvial and Pluvial Areas using RAS-Commander

We will leverage the HEC RAS Summary Outputs to delineate the Fluvial and Pluvial Areas

Maximum Water Surface Elevation (WSEL) for each cell is recorded, along with the timestamps of when the maximum WSEL occurs.

By locating adjacent cells with dissimilar timestamps, we can delineate the Fluvial and Pluvial Areas.




A note about datframe types: 

Information from the HEC-RAS plan files are generally dataframes.  The text file interface is for the 32-bit side of HEC-RAS and all spatial data is most easily accessed in the HDF files.  This includes plan_df, geom_df, hdf_paths_df

Geometry elements (Mesh Faces and Nodes) are provided as Geodataframes (cell_polygons_gdf, boundary_gdf)


## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [2]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [3]:
# Import all required modules
#from ras_commander import *  # Import all ras-commander modules

# Import the required libraries for this notebook
import h5py
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from pathlib import Path

In [4]:
# Download the BaldEagleCrkMulti2D project from HEC and run plan 06

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
bald_eagle_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = bald_eagle_path / "BaldEagleDamBrk.p06.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    RasExamples.extract_project(["BaldEagleCrkMulti2D"])


    # Initialize the RAS project using the default global ras object
    init_ras_project(bald_eagle_path, "6.6")
    logging.info(f"Bald Eagle project initialized with folder: {ras.project_folder}")
    
    logging.info(f"Bald Eagle object id: {id(ras)}")
    
    # Define the plan number to execute
    plan_number = "06"

    # Update the run flags in the plan file
    RasPlan.update_run_flags(
        plan_number,
        geometry_preprocessor=True,  # Run HTab
        unsteady_flow_simulation=True,  # Run UNet
        post_processor=True,  # Run PostProcess
        floodplain_mapping=False,  # Run RASMapper
    )

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("BaldEagleCrkMulti2D.p06.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the default global ras object
    init_ras_project(bald_eagle_path, "6.6")
    plan_number = 6
your_project_path = bald_eagle_path

2025-12-02 20:12:29 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.rasmap


BaldEagleCrkMulti2D.p06.hdf already exists. Skipping project extraction and plan execution.


###  OPTIONAL: Use your own project instead

your_project_path = Path(r"D:\yourprojectpath")

init_ras_project(your_project_path, "6.6")
plan_number = "01"  # Plan number to use for this notebook 



### If you use this code cell, don't run the previous cell or change to markdown
### NOTE: Ensure the HDF Results file was generated by HEC-RAS Version 6.x or above

# Explore Project Dataframes using 'ras' Object

In [5]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for bald_eagle project
print("Plan DataFrame for bald_eagle project:")
ras.plan_df

Plan DataFrame for bald_eagle project:


['   plan_number unsteady_number geometry_number  \\\n', '0           13              07              06   \n', '1           15              12              08   \n', '2           17              09              10   \n', '3           18              10              11   \n', '4           19              11              12   \n', '5           03              13              09   \n', '6           04              01              13   \n', '7           02              01              01   \n', '8           01              01              01   \n', '9           05              02              03   \n', '10          06              03              09   \n', '\n', '                                 Plan Title Program Version  \\\n', '0                   PMF with Multi 2D Areas            5.10   \n', '1               1d-2D Dambreak Refined Grid            5.10   \n', '2                           2D to 1D No Dam            5.00   \n', '3                              2D to 2D Run            5.0
...
[Output truncated, 7422 characters total]

In [6]:
print("\nGeometry DataFrame for the project:")
ras.geom_df


Geometry DataFrame for the project:


['  geom_file geom_number                                          full_path  \\\n', '0       g06          06  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '1       g08          08  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '2       g10          10  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '3       g11          11  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '4       g12          12  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '5       g09          09  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '6       g13          13  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '7       g01          01  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '8       g03          03  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '9       g02          02  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '\n', '                                            hdf_path  \n', '0  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '1  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '2  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '3  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '4  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '5  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '6  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '7  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '8  C:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '9  C:\\GH\\ras-commander\\examples\\example_projects\\...  ']

In [7]:
print("\nUnsteady DataFrame for the project:")
ras.unsteady_df


Unsteady DataFrame for the project:


['  unsteady_number                                          full_path  \\\n', '0              07  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '1              08  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '2              09  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '3              10  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '4              11  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '5              12  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '6              13  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '7              01  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '8              02  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '9              03  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '\n', '                             Flow Title Program Version Use Restart  \\\n', '0               P
...
[Output truncated, 4267 characters total]

In [8]:
print("\nBoundary Conditions DataFrame for the project:")
ras.boundaries_df 


Boundary Conditions DataFrame for the project:


['   unsteady_number  boundary_condition_number river_reach_name river_station  \\\n', '0               07                          1   Bald Eagle Cr.    Lock Haven   \n', '1               07                          2   Bald Eagle Cr.    Lock Haven   \n', '2               07                          3   Bald Eagle Cr.    Lock Haven   \n', '3               07                          4   Bald Eagle Cr.    Lock Haven   \n', '4               07                          5   Bald Eagle Cr.    Lock Haven   \n', '5               07                          6   Bald Eagle Cr.    Lock Haven   \n', '6               07                          7   Bald Eagle Cr.    Lock Haven   \n', '7               07                          8   Bald Eagle Cr.    Lock Haven   \n', '8               07                          9   Bald Eagle Cr.    Lock Haven   \n', '9               07                         10   Bald Eagle Cr.    Lock Haven   \n', '10              08                          1   Bald Eagle Cr.
...
[Output truncated, 29090 characters total]

# Find Paths for Results and Geometry HDF's

In [9]:
plan_number = "06"
print(plan_number)

# Get the plan HDF path for the plan_number defined above
plan_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

06


In [10]:
plan_hdf_path

["'C:\\\\GH\\\\ras-commander\\\\examples\\\\example_projects\\\\BaldEagleCrkMulti2D\\\\BaldEagleDamBrk.p06.hdf'"]

In [11]:
# Get the geometry HDF path
geom_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'Geom Path'].values[0] + '.hdf'

In [12]:
geom_hdf_path


["'C:\\\\GH\\\\ras-commander\\\\examples\\\\example_projects\\\\BaldEagleCrkMulti2D\\\\BaldEagleDamBrk.g09.hdf'"]

In [13]:
print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")


Plan HDF path for Plan 06: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
Geometry HDF path for Plan 06: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf


# Fluvial Pluvial Delineation using RAS-Commander 

Using the Maximum WSE Results layer, which contains the maximum water surface and time stamp of the maximum water surface, mesh cell faces are categorized.  If the difference in time (delta_t) in hours is greater than the (user defined, default 12) duration specified, that mesh cell face is added to the fluvial-pluvial boundary dataset. 

This is meant to provide a draft fluvial-pluvial boundary for floodplain analysis, to the extent it can be derived directly from the HEC-RAS results files. 

The function attempts to combine adjacent line segments to simplify the resulting geometry, but GIS cleanup and manual interpolation will be required to create a closed polygon boundary that could be used for further processing steps.  However, this approach does provide an efficient method for providing a draft boundary that is based on HEC-RAS's direct computations and mesh cell faces. 

In [14]:
# Using mesh_max_ws, get the cell coordinates and plot the max water surface as a map
import matplotlib.pyplot as plt
from ras_commander import HdfMesh
from ras_commander import HdfResultsMesh
from shapely.geometry import Point

# Get mesh max water surface
max_ws_df = HdfResultsMesh.get_mesh_max_ws(plan_hdf_path)

print("max_ws_df")
print(max_ws_df)

# If you get an error here, you may have a pre-6.0 HDF.  Re-run in 6.x to generate a new results file

2025-12-02 20:12:29 - ras_commander.hdf.HdfResultsMesh - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:29 - ras_commander.hdf.HdfResultsMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:29 - ras_commander.hdf.HdfResultsMesh - INFO - Processing summary output for variable: Maximum Water Surface
2025-12-02 20:12:29 - ras_commander.hdf.HdfMesh - INFO - Using HDF file from h5py.File object: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:29 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:29 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\

max_ws_df
         mesh_name  cell_id  maximum_water_surface maximum_water_surface_time  \
0      BaldEagleCr        0             704.054443        2018-09-10 18:00:00   
1      BaldEagleCr        1             692.377991        2018-09-10 18:04:00   
2      BaldEagleCr        2             671.183472        2018-09-10 18:13:20   
3      BaldEagleCr        3             660.605469        2018-09-10 18:54:40   
4      BaldEagleCr        4             660.586243        2018-09-10 18:55:20   
...            ...      ...                    ...                        ...   
19592  BaldEagleCr    19592               0.000000        2018-09-09 00:00:00   
19593  BaldEagleCr    19593               0.000000        2018-09-09 00:00:00   
19594  BaldEagleCr    19594               0.000000        2018-09-09 00:00:00   
19595  BaldEagleCr    19595               0.000000        2018-09-09 00:00:00   
19596  BaldEagleCr    19596               0.000000        2018-09-09 00:00:00   

                 

In [15]:
# Call the function to plot
HdfResultsPlot.plot_results_max_wsel(max_ws_df)

# Plot the time of maximum water surface elevation
HdfResultsPlot.plot_results_max_wsel_time(max_ws_df)

# Print the first few rows of the merged dataframe for verification
print("\nFirst few rows of the merged dataframe:")
max_ws_df


<Figure size 1200x800 with 2 Axes>

<Figure size 1200x800 with 2 Axes>


Simulation Start Time: 2018-09-09 00:00:00
Time Range: 120.0 hours

Timing Statistics (hours since start):
count    19597.000000
mean        63.225584
std         40.179644
min          0.000000
25%         34.000000
50%         42.700000
75%        115.677778
max        120.000000
Name: max_wsel_time, dtype: float64

First few rows of the merged dataframe:


['         mesh_name  cell_id  maximum_water_surface maximum_water_surface_time  \\\n', '0      BaldEagleCr        0             704.054443        2018-09-10 18:00:00   \n', '1      BaldEagleCr        1             692.377991        2018-09-10 18:04:00   \n', '2      BaldEagleCr        2             671.183472        2018-09-10 18:13:20   \n', '3      BaldEagleCr        3             660.605469        2018-09-10 18:54:40   \n', '4      BaldEagleCr        4             660.586243        2018-09-10 18:55:20   \n', '...            ...      ...                    ...                        ...   \n', '19592  BaldEagleCr    19592               0.000000        2018-09-09 00:00:00   \n', '19593  BaldEagleCr    19593               0.000000        2018-09-09 00:00:00   \n', '19594  BaldEagleCr    19594               0.000000        2018-09-09 00:00:00   \n', '19595  BaldEagleCr    19595               0.000000        2018-09-09 00:00:00   \n', '19596  BaldEagleCr    19596               0.000000 
...
[Output truncated, 2376 characters total]

In [16]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Projection: {projection}")
else:
    print("No projection information found.")

2025-12-02 20:12:30 - ras_commander.hdf.HdfBase - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfBase - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfBase - INFO - Found projection in HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Extracting Projection from HDF
Projection: PROJCS["NAD_1983_StatePlane_Pennsylvania_North_FIPS_3701_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-77.75],PARAMETER["Standard_Parallel_1",40.88333333333333],PARAMETER["Standard_Parallel_2",41.95],PARAMETER["Latitude_Of_Origin",40.16666666666666],UNIT["Foot_US",0.3048006096012192]]


In [17]:
# Example: Extract Cell Polygons
print("\nExample 6: Extracting Cell Polygons")
cell_polygons_gdf = HdfMesh.get_mesh_cell_polygons(geom_hdf_path)


# Call the function to plot cell polygons
#cell_polygons_gdf = HdfFluvialPluvial.plot_cell_polygons(cell_polygons_gdf, projection)


2025-12-02 20:12:30 - ras_commander.hdf.HdfMesh - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:30 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-com


Example 6: Extracting Cell Polygons


2025-12-02 20:12:31 - ras_commander.hdf.HdfBase - INFO - Using HDF file from h5py.File object: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfBase - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfBase - INFO - Found projection in HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfBase - INFO - Using HDF file from h5py.File object: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfBase - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfBase - INFO - Found projection in HDF file: C:\GH\ras-commande

In [18]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Polygon, MultiLineString
from collections import defaultdict
from tqdm import tqdm
from rtree import index


# Example usage:
boundary_gdf = HdfFluvialPluvial.calculate_fluvial_pluvial_boundary(plan_hdf_path, delta_t=72)

2025-12-02 20:12:31 - ras_commander.hdf.HdfFluvialPluvial - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfFluvialPluvial - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfFluvialPluvial - INFO - Getting cell polygons from HDF file...
2025-12-02 20:12:31 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:31 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMul

In [19]:
# Calculate statistics about the boundary line lengths
boundary_lengths = boundary_gdf.geometry.length

print("Boundary line length statistics:")
print(f"Max length: {boundary_lengths.max():.2f}")
print(f"Min length: {boundary_lengths.min():.2f}")
print(f"Average length: {boundary_lengths.mean():.2f}")
print(f"Median length: {boundary_lengths.median():.2f}")

# Print general information about the boundary GeoDataFrame
print("\nBoundary GeoDataFrame info:")
print(boundary_gdf.info())

Boundary line length statistics:
Max length: 441.63
Min length: 1.57
Average length: 242.54
Median length: 250.00

Boundary GeoDataFrame info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3201 entries, 0 to 3200
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  3201 non-null   geometry
dtypes: geometry(1)
memory usage: 25.1 KB
None


In [20]:
# Visualize the results
fig, ax = plt.subplots(figsize=(12, 8))
cell_polygons_gdf.plot(ax=ax, edgecolor='gray', facecolor='none', alpha=0.5)
boundary_gdf.plot(ax=ax, color='red', linewidth=2)
plt.title('Fluvial-Pluvial Boundary')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

<Figure size 1200x800 with 1 Axes>

In [21]:
length_threshold = 3000 #in same units as X and Y coordinates

# Filter out boundary lines below the length threshold
filtered_boundary_gdf = boundary_gdf[boundary_lengths >= length_threshold]
highlighted_boundary_gdf = boundary_gdf[boundary_lengths < length_threshold]

# Visualize the results with highlighted boundaries below the threshold
fig, ax = plt.subplots(figsize=(12, 8))
cell_polygons_gdf.plot(ax=ax, edgecolor='gray', facecolor='none', alpha=0.5)
filtered_boundary_gdf.plot(ax=ax, color='red', linewidth=2, label='Valid Boundaries')
highlighted_boundary_gdf.plot(ax=ax, color='blue', linewidth=2, linestyle='--', label='Highlighted Boundaries Below Threshold')
plt.title('Fluvial-Pluvial Boundary with Length Threshold')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.legend()
plt.show()


C:\Users\billk_clb\AppData\Local\Temp\ipykernel_57556\1399630256.py:10: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  filtered_boundary_gdf.plot(ax=ax, color='red', linewidth=2, label='Valid Boundaries')


<Figure size 1200x800 with 1 Axes>

In [22]:
# Example usage using Optional min_line_length argument:
boundary_gdf = HdfFluvialPluvial.calculate_fluvial_pluvial_boundary(plan_hdf_path, delta_t=72, min_line_length=3000)

2025-12-02 20:12:39 - ras_commander.hdf.HdfFluvialPluvial - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:39 - ras_commander.hdf.HdfFluvialPluvial - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:39 - ras_commander.hdf.HdfFluvialPluvial - INFO - Getting cell polygons from HDF file...
2025-12-02 20:12:39 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:39 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:39 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMul

In [23]:
# To get the classified flood zones as polygons:
flood_polygons_gdf = HdfFluvialPluvial.generate_fluvial_pluvial_polygons(
    plan_hdf_path, 
    delta_t=10, 
    temporal_tolerance_hours=1.0,
    min_polygon_area_acres=1000
)

flood_polygons_gdf

# Plot the classified flood zones as polygons, colored by classification
fig, ax = plt.subplots(figsize=(12, 8))
flood_polygons_gdf.plot(
    ax=ax,
    column="classification",
    categorical=True,
    legend=True,
    edgecolor="black",
    alpha=0.7
)
plt.title("Fluvial-Pluvial Classified Flood Zones")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


2025-12-02 20:12:43 - ras_commander.hdf.HdfFluvialPluvial - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:43 - ras_commander.hdf.HdfFluvialPluvial - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:43 - ras_commander.hdf.HdfFluvialPluvial - INFO - Loading mesh and results data...
2025-12-02 20:12:43 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:43 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:43 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\B

<Figure size 1200x800 with 1 Axes>

In [24]:
# To get the classified flood zones as polygons:
flood_polygons_gdf = HdfFluvialPluvial.generate_fluvial_pluvial_polygons(
    plan_hdf_path, 
    delta_t=12, 
    temporal_tolerance_hours=1.0,
    min_polygon_area_acres=200
)

flood_polygons_gdf

# Plot the classified flood zones as polygons, colored by classification
fig, ax = plt.subplots(figsize=(12, 8))
flood_polygons_gdf.plot(
    ax=ax,
    column="classification",
    categorical=True,
    legend=True,
    edgecolor="black",
    alpha=0.7
)
plt.title("Fluvial-Pluvial Classified Flood Zones")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


2025-12-02 20:12:49 - ras_commander.hdf.HdfFluvialPluvial - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:49 - ras_commander.hdf.HdfFluvialPluvial - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:49 - ras_commander.hdf.HdfFluvialPluvial - INFO - Loading mesh and results data...
2025-12-02 20:12:49 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:49 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:49 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\B

<Figure size 1200x800 with 1 Axes>

In [25]:
# To get the classified flood zones as polygons:
flood_polygons_gdf = HdfFluvialPluvial.generate_fluvial_pluvial_polygons(
    plan_hdf_path, 
    delta_t=14, 
    temporal_tolerance_hours=1.0,
    min_polygon_area_acres=1000
)

flood_polygons_gdf

# Plot the classified flood zones as polygons, colored by classification
fig, ax = plt.subplots(figsize=(12, 8))
flood_polygons_gdf.plot(
    ax=ax,
    column="classification",
    categorical=True,
    legend=True,
    edgecolor="black",
    alpha=0.7
)
plt.title("Fluvial-Pluvial Classified Flood Zones")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


2025-12-02 20:12:55 - ras_commander.hdf.HdfFluvialPluvial - INFO - Using HDF file from direct string path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:55 - ras_commander.hdf.HdfFluvialPluvial - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:55 - ras_commander.hdf.HdfFluvialPluvial - INFO - Loading mesh and results data...
2025-12-02 20:12:55 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:55 - ras_commander.hdf.HdfMesh - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-12-02 20:12:55 - ras_commander.hdf.HdfMesh - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\B

<Figure size 1200x800 with 1 Axes>

In [26]:
flood_polygons_gdf.to_file(driver='GeoJSON', filename='flood_polygons.geojson')

2025-12-02 20:13:00 - pyogrio._io - INFO - Created 3 records


In [27]:
# Create fluvial_pluvial_boundary subfolder
output_dir = your_project_path / "fluvial_pluvial_boundary"
output_dir.mkdir(exist_ok=True)
print(f"Output directory created/verified at: {output_dir}")

# Save to GeoJSON in output directory
boundary_gdf.to_file(output_dir / 'fluvial_pluvial_boundary.geojson', driver='GeoJSON')

2025-12-02 20:13:00 - pyogrio._io - INFO - Created 0 records


Output directory created/verified at: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\fluvial_pluvial_boundary
